<a href="https://colab.research.google.com/github/tylerb55/COMP530/blob/main/ResNetxMobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/tylerb55/COMP530.git

fatal: destination path 'COMP530' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
import collections
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import PIL.Image as Image

# **Split the train test and validation set**

In [3]:
def Train_Validation_Test_Set(directory_name):
  """a function to load the images in a large directory into a tensorflow dataset object
  the data is split 80:10:10 in training:validation:test. The dataset is shuffled before splitting
  and the images are formatted to 512x512 pixels and grayscale(one color channel and the values range from 0 to 255)"""
  dataset_train=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='training',
                                                                    batch_size=None
                                                                    )

  dataset_validation=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.2,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
  
  dataset_test=tf.keras.preprocessing.image_dataset_from_directory('/content/COMP530/'+directory_name,
                                                                    labels='inferred',
                                                                    label_mode='int',
                                                                    class_names=['NormalCases','cancercases'],
                                                                    image_size=(512,512),
                                                                    shuffle=True,
                                                                    seed=305,
                                                                    validation_split=0.1,
                                                                    subset='validation',
                                                                    batch_size=None
                                                                    )
    
  dataset_validation=dataset_validation.take(dataset_test.__len__())

  return dataset_train,dataset_validation,dataset_test

In [4]:
def ResNetPreprocess(image,label):
  image = image/255.0
  # Resize images from 512x512 to 224x224
  image = tf.image.resize(image, (224,224))
  return image, label

In [5]:
BATCH_SIZE=32

dataset_train,dataset_validation, dataset_test=Train_Validation_Test_Set("Dataset1")
dataset_train=dataset_train.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_validation=dataset_validation.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)
dataset_test=dataset_test.map(ResNetPreprocess).batch(batch_size=BATCH_SIZE,drop_remainder=True)

AUTOTUNE = tf.data.AUTOTUNE
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_validation = dataset_validation.cache().prefetch(buffer_size=AUTOTUNE)

Found 1097 files belonging to 2 classes.
Using 878 files for training.
Found 1097 files belonging to 2 classes.
Using 219 files for validation.
Found 1097 files belonging to 2 classes.
Using 109 files for validation.


# **mobilenet v2**

In [6]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_model = mobilenet_v2

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

num_classes = 1

mobilenet = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes,activation='sigmoid')
])

mobilenet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
mobilenet.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 30

history = mobilenet.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/30
27/27 [==============================] - 4s 52ms/step - loss: 0.1773 - acc: 0.6215 - val_loss: 0.2545 - val_acc: 0.6042
Epoch 2/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1752 - acc: 0.6215 - val_loss: 0.2530 - val_acc: 0.6042
Epoch 3/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1732 - acc: 0.6215 - val_loss: 0.2516 - val_acc: 0.6042
Epoch 4/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1712 - acc: 0.6215 - val_loss: 0.2501 - val_acc: 0.6042
Epoch 5/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1693 - acc: 0.6215 - val_loss: 0.2487 - val_acc: 0.6042
Epoch 6/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1674 - acc: 0.6215 - val_loss: 0.2474 - val_acc: 0.6042
Epoch 7/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1656 - acc: 0.6215 - val_loss: 0.2460 - val_acc: 0.6042
Epoch 8/30
27/27 [==============================] - 1s 28ms/step - loss: 0.1638 - a

In [8]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

KeyError: ignored

# **Resnet 50**

In [16]:
resnet = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
feature_extractor_model = resnet

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

num_classes = 1

resnet50 = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes,activation='sigmoid')
])

resnet50.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [17]:
resnet50.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.FalseNegatives()])

NUM_EPOCHS = 10

history = resnet50.fit(dataset_train,
                    validation_data=dataset_validation,
                    epochs=NUM_EPOCHS)

Epoch 1/10
27/27 [==============================] - 9s 105ms/step - loss: 0.4551 - binary_accuracy: 0.7650 - precision_1: 0.8047 - val_loss: 0.4348 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8387
Epoch 2/10
27/27 [==============================] - 2s 68ms/step - loss: 0.3064 - binary_accuracy: 0.8773 - precision_1: 0.9168 - val_loss: 0.3896 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8667
Epoch 3/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2700 - binary_accuracy: 0.8900 - precision_1: 0.9438 - val_loss: 0.3471 - val_binary_accuracy: 0.8333 - val_precision_1: 0.8621
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2452 - binary_accuracy: 0.8981 - precision_1: 0.9517 - val_loss: 0.3243 - val_binary_accuracy: 0.8542 - val_precision_1: 0.8793
Epoch 5/10
27/27 [==============================] - 2s 66ms/step - loss: 0.2264 - binary_accuracy: 0.8970 - precision_1: 0.9534 - val_loss: 0.3080 - val_binary_accuracy: 0.8542 - val_prec

In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('training accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
resnet50.evaluate(dataset_test)

In [ ]:
mobilenet.evaluate(dataset_test)